# LLM from scratch: vector go brrr

In [1]:
from __future__ import annotations


def _add(a: Tensor, b: Tensor):
    """
    Add two tensors
    """
    result = Tensor(a.value + b.value)
    result.local_derivatives = (Tensor(1), Tensor(1))
    result.args = (a, b)
    return result


def _sub(a: Tensor, b: Tensor):
    """
    Subtract tensor b from a
    """
    result = Tensor(a.value - b.value)
    result.local_derivatives = (Tensor(1), Tensor(-1))
    result.args = (a, b)
    return result


def _mul(a: Tensor, b: Tensor):
    """
    Multiply two tensors
    """
    result = Tensor(a.value * b.value)
    result.local_derivatives = (b, a)
    result.args = (a, b)
    return result


class Tensor:
    """
    A float that can be differentiated
    """

    args: tuple[Tensor] = ()
    derivative_fns = ()
    # The derivative (once we've calculated it).  This is None if the derivative
    # has not been computed yet
    derivative: Tensor | None = None

    def __init__(self, value: float):
        self.value = value

    def __repr__(self) -> str:
        return f"Tensor({self.value.__repr__()})"

    def __eq__(self, other) -> bool:
        if not isinstance(other, Tensor):
            raise TypeError(f"Cannot compare a Tensor with a {type(other)}")
        return self.value == other.value

    def __add__(self, other) -> Tensor:
        if not isinstance(other, Tensor):
            raise TypeError(f"Cannot add a Tensor to a {type(other)}")
        return _add(self, other)

    def __sub__(self, other) -> Tensor:
        if not isinstance(other, Tensor):
            raise TypeError(f"Cannot subtract a Tensor from a {type(other)}")
        return _sub(self, other)

    def __mul__(self, other) -> Tensor:
        if not isinstance(other, Tensor):
            raise TypeError(f"Cannot multiply a Tensor with a {type(other)}")
        return _mul(self, other)

    def __iadd__(self, other) -> Tensor:
        return self.__add__(other)

    def __isub__(self, other) -> Tensor:
        return self.__sub__(other)

    def __imul__(self, other) -> Tensor:
        return self.__mul__(other)

    def __repr__(self) -> str:
        return f"Tensor({self.value})"

    def backward(self):
        if self.args is None or self.local_derivatives is None:
            raise ValueError(
                "Cannot differentiate a Tensor that is not a function of other Tensors"
            )

        stack = [(self, Tensor(1))]

        while stack:
            node, current_derivative = stack.pop()

            # if we have reached a parameter (it has no arguments
            # because it wasn't created by an operation) then add the
            # current_derivative to derivative
            if not node.args:
                if node.derivative is None:
                    node.derivative = current_derivative
                else:
                    node.derivative += current_derivative
                continue

            for arg, derivative in zip(node.args, node.local_derivatives):
                stack.append((arg, current_derivative * derivative))

Last time, we built the core of deep learning system: the automatic differentiation engine. It can differentiate any function you like, provided you only like doing addition, subtraction, multiplication and division. 

In theory, this is most of what we need. We can add a few new functions like `log` and `exp` and we'll have all of the building blocks that we need to do the calculations for a language model. Lets try adding the most important operation first: matrix multiplication. 

In [2]:
import random

def matrix_add(
    matrix_1: list[list[Tensor]], 
    matrix_2: list[list[Tensor]], 
    ) -> list[list[Tensor]]:

    # check that the shapes match
    height_1, width_1 = len(matrix_1), len(matrix_1[0])
    height_2, width_2 = len(matrix_2), len(matrix_2[0])

    assert width_1 == height_2

    out = [[Tensor(0) for _ in range(width_2)] for _ in range(height_1)]

    for i in range(height_1):
        for j in range(width_2):
            out[i][j] = matrix_1[i][j] + matrix_2[i][j]

    return out

size = 100

matrix_1 = [[Tensor(random.random()) for _ in range(size)] for _ in range(size)]
matrix_2 = [[Tensor(random.random()) for _ in range(size)] for _ in range(size)]

result = matrix_add(matrix_1, matrix_2)

print(result[0][:3])

[Tensor(0.9687713981380367), Tensor(1.1881056150650755), Tensor(0.8222890444360291)]


This gives us the right answer, but there is a problem: it is really expensive.
Lets re-run this calculation and track the CPU and RAM usage.

In [3]:
import time
from humanize import naturalsize, naturaldelta
# tracks memory usage
from pympler import asizeof

matrix_1 = [[Tensor(random.random()) for _ in range(1000)] for _ in range(1000)]
matrix_2 = [[Tensor(random.random()) for _ in range(1000)] for _ in range(1000)]

start = time.time()
before_memory_usage = asizeof.asizeof(matrix_1) + asizeof.asizeof(matrix_2)

result = matrix_add(matrix_1, matrix_2)

duration = time.time() - start
memory_usage = asizeof.asizeof(result) +asizeof.asizeof(matrix_1) + asizeof.asizeof(matrix_2)
memory_usage -= before_memory_usage

print(f'Duration: {naturaldelta(duration, minimum_unit="milliseconds")}')
print(f'Memory usage: {naturalsize(memory_usage)}')

KeyboardInterrupt: 

In [4]:
import numpy as np
a = np.random.random((1000,1000))
b = np.random.random((1000,1000))

c = a + b

asizeof.asizeof(c)

8000144

In deep learning terms, this matrix is tiny. Modern models are usually measured by how many billions of parameters they contain while this matrix has 20,000. We want our model to produce multiple tokens per second which means performing 1000s (depending on the model) of matrix mutliplications per second. Clearly, if we want to build something that fits in memory and trains before the heat death of the universe, some optimisations are needed.

Our current approach has some downsides:
 - Each parameter requires a lot of memory to store
 - We are using python loops which are very slow

That said, any improvement will need to keep the parts of our tensor object that work well. Namely, the automatic differentiation and nice pythonic interface.

The answer, as is often the case when trying to make python programs fast, is to use Numpy.

## Numpy

Technically, I'm breaking my own rules by using numpy. I promised in the first blog post that the LLM would be built in "vanilla" python. Numpy is not included in the python standard library so it does not count as "vanilla", in some senses of the word. However, it is used to ubiquitously in the python ecosystem that it is vanilla in all but name. Regardless, if I'm not allowed to use numpy then I'd have to effectively recreate it and that would be a waste of everyone's time.

Using numpy fixes both of the issues with our current approach. It is very memory efficient and is dramatically faster than doing computations in pure python. If we want to use it, we'll need to figure out how to include numpy arrays into our tensor objects. When using numpy, you do operations on arrays of values instead of on individual values. As a first step, lets get our `Tensor` to use numpy arrays instead of python floats. (We can also rename some of the variables while we're here)

In [5]:
class Tensor:
    """
    An array of numbers that can be differentiated
    """
    args: tuple[Tensor] = ()
    back_fns: tuple = ()
    grad: Tensor | None = None
    array: np.ndarray

    def __init__(self, array: np.ndarray):
        self.array = array

So far, so good, but what happens to our operations? Originally, we did addition like this:

In [6]:
def _add(a: Tensor, b: Tensor):
    """
    Add two tensors
    """
    result = Tensor(a.value + b.value)
    result.back_fns = (Tensor(1), Tensor(1))
    result.args = (a, b)
    return result

Now we need to make a couple of changes. First of all, we'll need to change `.value` to `.array`. Provided they are the same shape, you can add numpy arrays in the way as you add python objects so we can leave the `+` as it is. Multiplying a matrix by 1 is the same as doing nothing so we can create a function that does nothing. 

In [9]:
def nothing(tensor: Tensor) -> Tensor:
    return tensor

def add(tensor_1: Tensor, tensor_2: Tensor) -> Tensor:
    """
    Add two tensors
    """
    result = Tensor(tensor_1.array + tensor_2.array)
    result.back_fns = (nothing, nothing)
    result.args = (tensor_1, tensor_2)
    return result

Lets try it out and see how much more efficient things are.

In [12]:
matrix_1 = Tensor(np.random.random((1000,1000))) 
matrix_2 = Tensor(np.random.random((1000,1000))) 

start = time.time()
before_memory_usage = asizeof.asizeof(matrix_1) + asizeof.asizeof(matrix_2)

result = add(matrix_1, matrix_2)

duration = time.time() - start
memory_usage = asizeof.asizeof(result) +asizeof.asizeof(matrix_1) + asizeof.asizeof(matrix_2)
memory_usage -= before_memory_usage

print(f'Duration: {naturaldelta(duration, minimum_unit="milliseconds")}')
print(f'Memory usage: {naturalsize(memory_usage)}')

Duration: 0 milliseconds
Memory usage: 24.0 MB


That is the easy part. Things get a bit harder when we look at the differentiation. 

# TODO: Intro

One way we could do this is by building a list of every operation we need and then using linear algebra to figure out the derivative for each operation. This works, but has a number of problems:
 - It takes ages and I don't have the time to plow through all of those derivatives
 - Matrix derivatives can be tricky. For example you need to care about row vs column vectors
 - The equations very quickly get long and complex which is both frustrating to deal with and error prone

Instead, there is a much more elegant solution: the einsum operation.

## Ein-what?
Standard linear algebra notation usually looks something like this:
$$\textbf{z} = c \cdot \textbf{x}^{T}A\textbf{x}$$

You can tell what type of object a given symbol represents by what it looks like:
 - scalars are normal text
 - vectors are bold
 - matrices are capitalised

If two symbols are next to each other, then they are matrix multiplied together. Because matrix multiplication depends on the shape of each symbol, we have to keep track of whether a vector is horizontal or vertical (row or column) to make sure that each matrix multiplication is allowed.

This system is usable, but difficult. The normal rules of high-school algebra don't always apply and you have to be a lot more careful about what you are doing. When we start including derivatives, this only gets harder.

Instead, there is an alternative system which end up being a lot simpler and easier to use.

We can imagine giving each element in an object an index (which row is it on, which column etc)
$$\begin{pmatrix}
a_1\\ 
...\\ 
a_n
\end{pmatrix} = \textbf{a}$$

Then, we can invent a dummy variable that stands for one of these indices:
$$\begin{pmatrix}
a_1\\ 
...\\ 
a_n
\end{pmatrix} = \textbf{a} = a_{i}$$

Whenever we set $i$ to an actual value, we get one of the elements of the vector.
This works for tensors of any shape:
$$\begin{pmatrix}
a_1 & ... & a_n\\ 
\vdots &  \ddots & \vdots\\ 
z_1 & ... & z_n
\end{pmatrix} = B = b_{ij}$$
$$\begin{pmatrix}
(4,2,6)&(8,2,6)&(2,2,5)\\
(3,3,0)&(8,9,2)&(0,0,3)\\
(3,0,2)&(6,8,3)&(7,7,5)\\
\end{pmatrix} = C = c_{ijk}$$

Note that each dimension gets its own letter. It doesn't matter which letters you choose, provided each dimension has a different one.

This notation isn't very helpful on its own, but becomes very useful when we add some rules:
> If two objects have the same letter for their index, we multiply each value along these shared incides

For example, if we have:
$$\begin{pmatrix}
1&2&3\\
4&5&6
\end{pmatrix} = a_{ij}$$
and 
$$\begin{pmatrix}
1&2\\
3&4\\
5&6
\end{pmatrix} = b_{jk}$$
then
$$a_{ij}b_{jk} = $$

In [ ]:
import random
s = """"""
for row in range(1,4):
    column = []
    for col in range(1,4):
        item = []
        for idx in range(1,4):
            item.append(str(random.randint(0, 9)))
        item = f"({','.join(item)})"
        column.append(item)
    column = f"{'&'.join(column)}\\\\"
    print(column)

(4,2,6)&(8,2,6)&(2,2,5)\\
(3,3,0)&(8,9,2)&(0,0,3)\\
(3,0,2)&(6,8,3)&(7,7,5)\\


In [ ]:
import numpy as np

a = np.arange(6).reshape(3, 2)
b = np.arange(6, 12).reshape(2, 3)

np.einsum("ij,ji->j", a, b)

array([46, 94])

## TODO: examples of using einsum

# TODO: Midddle bit

Here is a tensor from last time

In [ ]:
from __future__ import annotations
class Tensor:
    """
    A float that can be differentiated
    """

    args: tuple[Tensor] = ()
    derivative_fns: tuple[Op] = ()
    # The derivative (once we've calculated it).  This is None if the derivative
    # has not been computed yet
    derivative: Tensor | None = None

    def __init__(self, value: float):
        self.value = value

    def __repr__(self) -> str:
        return f"Tensor({self.value.__repr__()})"

    def __eq__(self, other) -> bool:
        if not isinstance(other, Tensor):
            raise TypeError(f"Cannot compare a Tensor with a {type(other)}")
        return self.value == other.value

    def __add__(self, other) -> Tensor:
        if not isinstance(other, Tensor):
            raise TypeError(f"Cannot add a Tensor to a {type(other)}")
        return _add(self, other)

    def __sub__(self, other) -> Tensor:
        if not isinstance(other, Tensor):
            raise TypeError(f"Cannot subtract a Tensor from a {type(other)}")
        return _sub(self, other)

    def __mul__(self, other) -> Tensor:
        if not isinstance(other, Tensor):
            raise TypeError(f"Cannot multiply a Tensor with a {type(other)}")
        return _mul(self, other)

    def __iadd__(self, other) -> Tensor:
        return self.__add__(other)

    def __isub__(self, other) -> Tensor:
        return self.__sub__(other)

    def __imul__(self, other) -> Tensor:
        return self.__mul__(other)

    def __repr__(self) -> str:
        return f"Tensor({self.value})"

    def backward(self):
        if self.args is None or self.local_derivatives is None:
            raise ValueError(
                "Cannot differentiate a Tensor that is not a function of other Tensors"
            )

        stack = [(self, Tensor(1))]

        while stack:
            node, current_derivative = stack.pop()

            # if we have reached a parameter (it has no arguments
            # because it wasn't created by an operation) then add the
            # current_derivative to derivative
            if not node.args:
                if node.derivative is None:
                    node.derivative = current_derivative
                else:
                    node.derivative += current_derivative
                continue

            for arg, derivative in zip(node.args, node.local_derivatives):
                stack.append((arg, current_derivative * derivative))

We can no longer multiply values, instead we need to apply functions (to handle einsums)

In [ ]:
class Tensor:
    """
    A float that can be differentiated
    """

    args: tuple[Tensor] = ()
    derivative_fns: tuple[Op] = ()
    # The derivative (once we've calculated it).  This is None if the derivative
    # has not been computed yet
    derivative: Tensor | None = None

    def __init__(self, value: float):
        self.value = value

    def __repr__(self) -> str:
        return f"Tensor({self.value.__repr__()})"

    def __eq__(self, other) -> bool:
        if not isinstance(other, Tensor):
            raise TypeError(f"Cannot compare a Tensor with a {type(other)}")
        return self.value == other.value

    def __add__(self, other) -> Tensor:
        if not isinstance(other, Tensor):
            raise TypeError(f"Cannot add a Tensor to a {type(other)}")
        return _add(self, other)

    def __sub__(self, other) -> Tensor:
        if not isinstance(other, Tensor):
            raise TypeError(f"Cannot subtract a Tensor from a {type(other)}")
        return _sub(self, other)

    def __mul__(self, other) -> Tensor:
        if not isinstance(other, Tensor):
            raise TypeError(f"Cannot multiply a Tensor with a {type(other)}")
        return _mul(self, other)

    def __iadd__(self, other) -> Tensor:
        return self.__add__(other)

    def __isub__(self, other) -> Tensor:
        return self.__sub__(other)

    def __imul__(self, other) -> Tensor:
        return self.__mul__(other)

    def __repr__(self) -> str:
        return f"Tensor({self.value})"

    def backward(self):
        if self.args is None or self.local_derivatives is None:
            raise ValueError(
                "Cannot differentiate a Tensor that is not a function of other Tensors"
            )

        stack = [(self, Tensor(1))]

        while stack:
            node, current_derivative = stack.pop()

            # if we have reached a parameter (it has no arguments
            # because it wasn't created by an operation) then add the
            # current_derivative to derivative
            if not node.args:
                if node.derivative is None:
                    node.derivative = current_derivative
                else:
                    node.derivative += current_derivative
                continue

            for arg, derivative_fn in zip(node.args, node.derivative_fns):
                stack.append((arg, derivative_fn(current_derivative)))

This won't work because we haven't defined `_add`, `_sub` or `_mul`. Lets define them

### Basic Unary Ops

In [ ]:
def _mul(a: Tensor, b: Tensor):
    """
    Multiply two tensors
    """
    result = Tensor(a.value * b.value)
    result.local_derivatives = (b, a)
    result.args = (a, b)
    return result



def _umul(tensor: Tensor, constant: np.number) -> Tensor:
    assert isinstance(tensor, Tensor)
    assert np.isscalar(constant)

    result = Tensor(tensor.value * constant)
    result.derivative_fns = ()

    

In [ ]:
class einsum:
    def __init__(self, subscript: str):
        self.subscript = subscript
        self.indices, self.output = _parse_subscripts(subscript)

    def _generate_back_fns(self, tensors: Sequence[Tensor]):
        assert len(tensors) == len(self.indices)
        back_functions = []
        for idx in range(len(tensors)):

            def back_fn(tensor: Tensor, idx: int = idx):
                left_tensors = tensors[:idx]
                left_subscript = self.indices[:idx]

                right_tensors = tensors[idx + 1 :] if idx < len(tensors) - 1 else []
                right_subscript = (
                    self.indices[idx + 1 :] if idx < len(tensors) - 1 else []
                )

                subscript = left_subscript + [self.output] + right_subscript
                subscript = ",".join(subscript) + "->" + self.indices[idx]

                fn_args = [*left_tensors, tensor, *right_tensors]
                return einsum(subscript)(*fn_args)

            back_functions.append(back_fn)

        return back_functions

    def __call__(self, *tensors: Tensor):
        result = to_tensor(np.einsum(self.subscript, *tensors))
        result.args = tuple(tensors)
        result.back_fn = tuple(self._generate_back_fns(tensors))
        result.name = f"einsum {self.subscript}"
        return result

# The Proof

Let $A, B,...$ be a collection of tensors of arbitrary dimension.

Suppose these tensors are combined with a valid einsum operation as follows (in einstein notation):

$$z_{z_1, z_2, ...,z_n} = a_{a_1, a_2, ...,a_m}b_{b_1, b_2,...,b_p}...$$

Then, because the einsum operation is valid:

$$\{z_1, z_2, ...,z_n\} \subset \{a_1, a_2, ...,a_m,b_1, b_2,...,b_p,...\}$$

That is, every index in the output tensor is included at least once in the set of indices for input tensors.

Note here that while the set of indices for each individual tensor contains no duplicates: 
$$|\{z_1, z_2, ...,z_n\}| = n$$
$$|\{a_1, a_2, ...,a_m\}| = m$$
$$|\{b_1, b_2, ...,b_p\}| = p$$
The same index can appear in the set of indices for multiple input tensors.

The einsum operation can be written in terms of an `einsum` function as follows:

$$Z = \texttt{einsum}(a_1, a_2, ...,a_m,b_1, b_2,...,b_p,...\rightarrow z_1, z_2, ...,z_n)(A, B,...)$$


Now suppose that each input is potentially a function of some value $x$: $A(x), B(x), ...$ (although the derivative of each input wrt $x$ could be $0$) . In einstein notation, this becomes:

$$z_{z_1, z_2, ...,z_n}(x) = a_{a_1, a_2, ...,a_m}(x)b_{b_1, b_2,...,b_p}(x)...$$

Thus, we find 
$$\frac{\partial Z}{\partial x} = \frac{\partial z_{z_1, z_2, ...,z_n}(x)}{\partial x} = \frac{\partial }{\partial x}\left ( a_{a_1, a_2, ...,a_m}b_{b_1, b_2,...,b_p}... \right )$$

Because we are using einstein notation, we can apply the product rule to get:

$$\frac{\partial }{\partial x}\left ( a_{a_1, a_2, ...,a_m}b_{b_1, b_2,...,b_p}... \right ) = \frac{\partial a_{a_1, a_2, ...,a_m}(x)}{\partial x}b_{b_1, b_2,...,b_p}... + a_{a_1, a_2, ...,a_m}(x)\frac{\partial b_{b_1, b_2,...,b_p}}{\partial x}... + ...$$

Using the `einstein` function, we can rewrite this as

$$\frac{\partial Z}{\partial x}

# 

# TODO: Conclusion